<a href="https://colab.research.google.com/github/hongjinkong/opensw/blob/main/1119-3/deepfakedetection_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/mystlee/dfdc_deepfake_challenge.git

Cloning into 'dfdc_deepfake_challenge'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (65/65), done.


In [2]:
from google.colab import files
files.upload()  # 여기서 'kaggle.json' 파일을 선택하여 업로드


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hongjinkong","key":"4c9bd1c41ad59a0b429b20ee1998c074"}'}

In [3]:
import os

# Kaggle API 파일을 Colab에서 사용할 수 있는 위치로 이동
os.makedirs('/root/.kaggle', exist_ok=True)
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json  # 권한 설정


In [4]:
!kaggle datasets download -d manjilkarki/deepfake-and-real-images --force


Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
100% 1.68G/1.68G [00:21<00:00, 59.0MB/s]
100% 1.68G/1.68G [00:21<00:00, 85.2MB/s]


In [ ]:
!unzip deepfake-and-real-images.zip -d /content/deepfake_and_real_images


In [ ]:
import os

dataset_dir = '/content/deepfake_and_real_images'  # 데이터셋 경로
os.listdir(dataset_dir)


['Dataset']

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터셋 경로를 정확히 설정
dataset_dir = '/content/deepfake_and_real_images/Dataset'  # 실제 경로로 변경

# 이미지 데이터 생성기 (ImageDataGenerator)
datagen = ImageDataGenerator(
    rescale=1./255,          # 이미지 픽셀 값을 0~1로 정규화
    rotation_range=15,       # 이미지 회전
    width_shift_range=0.1,   # 가로 이동
    height_shift_range=0.1,  # 세로 이동
    horizontal_flip=True     # 수평 뒤집기
)

# train, validation, test 데이터 로드 (디렉토리 구조에 맞춰 자동으로 라벨링)
train_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Train'),  # 'Train' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',  # 다중 클래스 분류 (Fake: 0, Real: 1)
)

val_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Validation'),  # 'Validation' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)

test_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Test'),  # 'Test' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)


Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터셋 경로를 정확히 설정
dataset_dir = '/content/deepfake_and_real_images/Dataset'  # 실제 경로로 변경

# 이미지 데이터 생성기 (ImageDataGenerator)
datagen = ImageDataGenerator(
    rescale=1./255,          # 이미지 픽셀 값을 0~1로 정규화
    rotation_range=15,       # 이미지 회전
    width_shift_range=0.1,   # 가로 이동
    height_shift_range=0.1,  # 세로 이동
    horizontal_flip=True     # 수평 뒤집기
)

# train, validation, test 데이터 로드 (디렉토리 구조에 맞춰 자동으로 라벨링)
train_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Train'),  # 'Train' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',  # 다중 클래스 분류 (Fake: 0, Real: 1)
)

val_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Validation'),  # 'Validation' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)

test_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Test'),  # 'Test' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)

# 모델 정의 (이미지 분류 모델)
def build_encoder(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
    return Model(inputs, encoded, name="encoder")

def build_classifier_with_dropout(input_shape):
    encoder = build_encoder(input_shape)
    encoder_output = encoder.output
    flat = layers.Flatten()(encoder_output)
    dense = layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(flat)  # L2 정규화 추가
    dropout = layers.Dropout(0.5)(dense)  # 드롭아웃 추가
    output = layers.Dense(2, activation='softmax')(dropout)  # 두 클래스에 대한 softmax 활성화 함수
    classifier = Model(encoder.input, output, name="classifier_with_dropout")
    return classifier

# 모델 학습
input_shape = (128, 128, 3)
classifier = build_classifier_with_dropout(input_shape)
classifier.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 구조 출력
classifier.summary()

# 모델 학습
epochs = 50  # 에포크 수
classifier.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,  # validation 데이터로 모델 평가
)

# Grad-CAM heatmap을 계산하는 함수
def get_gradcam_heatmap(model, image, class_idx):
    last_conv_layer_name = None
    for layer in reversed(model.layers):
        if isinstance(layer, layers.Conv2D):
            last_conv_layer_name = layer.name
            break

    if not last_conv_layer_name:
        raise ValueError("No Conv2D layer found in the model.")

    last_conv_layer_model = Model(inputs=model.inputs, outputs=model.get_layer(last_conv_layer_name).output)
    classifier_model = Model(inputs=model.get_layer(last_conv_layer_name).output, outputs=model.outputs)

    image_input = np.expand_dims(image, axis=0)
    image_input = tf.convert_to_tensor(image_input, dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(image_input)
        conv_output = last_conv_layer_model(image_input)
        preds = classifier_model(conv_output)
        class_output = preds[:, class_idx]

    grads = tape.gradient(class_output, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_output = conv_output[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8

    return heatmap

# Grad-CAM + Bounding Box 시각화 함수
def display_gradcam_with_bbox(model, image, predicted_class, label, threshold=0.5):
    heatmap = get_gradcam_heatmap(model, image, predicted_class)

    if heatmap is None or np.all(heatmap == 0):
        print("No heatmap generated, skipping visualization.")
        return

    heatmap_resized = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(heatmap_color, 0.3, np.uint8(image * 255), 0.7, 0)

    ret, thresh = cv2.threshold(heatmap_resized, threshold * 255, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 100:
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(superimposed_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(superimposed_img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# Grad-CAM 시각화
test_image, label = next(test_generator)
predicted_class = np.argmax(classifier.predict(test_image), axis=-1)[0]  # 다중 클래스 분류에서 예측된 클래스 선택
display_gradcam_with_bbox(classifier, test_image[0], predicted_class, label, threshold=0.5)
print(f"Predicted Class: {predicted_class}, Label: {label}")


Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


Model: "classifier_with_dropout"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │       2,097,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,190,594 (8.36 MB)

 Trainable params: 2,190,594 (8.36 MB)

 Non-trainable params: 0 (0.00 B)

4375/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.6802 - loss: 0.6079

KeyboardInterrupt: 

In [7]:
#훈련용
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras import layers

# 데이터셋 경로를 정확히 설정
dataset_dir = '/content/deepfake_and_real_images/Dataset'  # 실제 경로로 변경

# 이미지 데이터 생성기 (ImageDataGenerator)
datagen = ImageDataGenerator(
    rescale=1./255,          # 이미지 픽셀 값을 0~1로 정규화
    rotation_range=15,       # 이미지 회전
    width_shift_range=0.1,   # 가로 이동
    height_shift_range=0.1,  # 세로 이동
    horizontal_flip=True     # 수평 뒤집기
)

# train, validation, test 데이터 로드 (디렉토리 구조에 맞춰 자동으로 라벨링)
train_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Train'),  # 'Train' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',  # 다중 클래스 분류 (Fake: 0, Real: 1)
)

# validation 데이터는 100개 샘플만 사용
val_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Validation'),  # 'Validation' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)

# test 데이터는 100개 샘플만 사용
test_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'Test'),  # 'Test' 폴더에 있는 데이터
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
)

# 훈련 데이터 일부만 사용: 500개 샘플만 사용
train_samples = 500  # 데이터의 일부만 사용
steps_per_epoch = train_samples // train_generator.batch_size  # 배치 크기에 맞게 steps 계산

# 모델 정의 (이미지 분류 모델)
def build_encoder(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
    return Model(inputs, encoded, name="encoder")

def build_classifier_with_dropout(input_shape):
    encoder = build_encoder(input_shape)
    encoder_output = encoder.output
    flat = layers.Flatten()(encoder_output)
    dense = layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(flat)  # L2 정규화 추가
    dropout = layers.Dropout(0.5)(dense)  # 드롭아웃 추가
    output = layers.Dense(2, activation='softmax')(dropout)  # 두 클래스에 대한 softmax 활성화 함수
    classifier = Model(encoder.input, output, name="classifier_with_dropout")
    return classifier

# 모델 학습
input_shape = (128, 128, 3)
classifier = build_classifier_with_dropout(input_shape)
classifier.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 구조 출력
classifier.summary()

# 모델 학습
epochs = 5  # 에포크 수를 적게 설정하여 테스트용으로 빠르게 실행
classifier.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,  # validation 데이터로 모델 평가
    steps_per_epoch=steps_per_epoch  # 훈련에 사용할 배치 수를 설정
)

# Grad-CAM heatmap을 계산하는 함수
def get_gradcam_heatmap(model, image, class_idx):
    last_conv_layer_name = None
    for layer in reversed(model.layers):
        if isinstance(layer, layers.Conv2D):
            last_conv_layer_name = layer.name
            break

    if not last_conv_layer_name:
        raise ValueError("No Conv2D layer found in the model.")

    last_conv_layer_model = Model(inputs=model.inputs, outputs=model.get_layer(last_conv_layer_name).output)
    classifier_model = Model(inputs=model.get_layer(last_conv_layer_name).output, outputs=model.outputs)

    image_input = np.expand_dims(image, axis=0)
    image_input = tf.convert_to_tensor(image_input, dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(image_input)
        conv_output = last_conv_layer_model(image_input)
        preds = classifier_model(conv_output)
        class_output = preds[:, class_idx]

    grads = tape.gradient(class_output, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_output = conv_output[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8

    return heatmap

# Grad-CAM + Bounding Box 시각화 함수
def display_gradcam_with_bbox(model, image, predicted_class, label, threshold=0.5):
    heatmap = get_gradcam_heatmap(model, image, predicted_class)

    if heatmap is None or np.all(heatmap == 0):
        print("No heatmap generated, skipping visualization.")
        return

    heatmap_resized = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    heatmap_color = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(heatmap_color, 0.3, np.uint8(image * 255), 0.7, 0)

    ret, thresh = cv2.threshold(heatmap_resized, threshold * 255, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 100:
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(superimposed_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(superimposed_img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# Grad-CAM 시각화
test_image, label = next(test_generator)
predicted_class = np.argmax(classifier.predict(test_image), axis=-1)[0]  # 첫 번째 이미지에 대해 예측
display_gradcam_with_bbox(classifier, test_image[0], predicted_class, label, threshold=0.5)
print(f"Predicted Class: {predicted_class}, Label: {label}")


Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


Model: "classifier_with_dropout"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │       2,097,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,190,594 (8.36 MB)

 Trainable params: 2,190,594 (8.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4944 - loss: 0.8930

KeyboardInterrupt: 